# Mapping Test Coverage in the Linux Kernel


## Will Beckett CSC 499 Honours Project

## Advised by Prof Daniel German


In [1]:
import pandas as pd
from pathlib import Path
import re
import subprocess
import numpy as np

import locations
from runner import tags

from ipywidgets import widgets, Layout, fixed
import plotly.express as px

In [2]:
non_static_df = pd.read_csv(locations.OUTDIR/"coverage_by_tag_ns_nt_i.csv").rename(
    columns={"non_static": "functions"}).fillna('')
static_df = pd.read_csv(locations.OUTDIR/"coverage_by_tag_s_nt_i.csv").fillna('')
non_static_df["ratio"] = non_static_df["tested"]/non_static_df["functions"]
static_df["ratio"] = static_df["tested"]/static_df["functions"]

In [3]:
def make_plot(df, directory):
    directory=directory.rstrip("/")
    ax = df[df["directory"] == directory].plot(
        x="tag", y=["functions", "tested", "ratio"], secondary_y="ratio", title=directory or 'whole kernel',
        figsize=(10, 10), fontsize=16
    )
    ax.set_xlabel("Version Number", fontdict={'fontsize':16})
    ax.set_ylabel("Number of Functions", fontdict={'fontsize':16})
    ax.right_ax.set_ylabel("Coverage Ratio", fontdict={'fontsize':16})
    bottom, top = ax.get_ylim()
    ax.set_ylim(0, top)
    bottom, top = ax.right_ax.get_ylim()
    ax.right_ax.set_ylim(0, top)
    
def interactive_plot(df):
    selector = widgets.Combobox(options=list(set(df["directory"])), description="directory",
                        layout=Layout(width="500px"))
    plot = widgets.interactive_output(make_plot, {'df': fixed(df), 'directory':selector})
    display(plot, selector)

In [4]:
interactive_plot(non_static_df)

Output()

Combobox(value='', description='directory', layout=Layout(width='500px'), options=('', 'drivers/staging/rtlwif…

In [5]:
interactive_plot(static_df)

Output()

Combobox(value='', description='directory', layout=Layout(width='500px'), options=('', 'drivers/net/ethernet/a…

Transitive tests are not counted.

In [ ]:
def test():
    function_a()
    
def function_a(): # this counts as tested
    funtion_b() 
    
def function_b(): # this does not count as tested
    pass 

In [6]:
def prep_treedf(df, version):
    treedf = df.copy()
    treedf = treedf[treedf.tag == version]
    treedf["directory"].replace("", "kernel root", inplace=True)

    treedf["parent"] = treedf["directory"].apply(get_parent)
    treedf["count excluding subdirectories"] = treedf["functions"]

    rows = treedf.copy().iterrows()
    for parent,value in treedf.groupby("parent")["functions"].sum().iteritems():
        treedf.loc[treedf["directory"] == parent,["count excluding subdirectories"]] -= value

    assert len(treedf[treedf["count excluding subdirectories"] < 0]) == 0
    return treedf

def treemap(df, version):
    treedf = prep_treedf(df, version)
    color = treedf["ratio"].replace(0, np.nan)
    fig = px.treemap(treedf, names="directory", parents="parent",
                     values="count excluding subdirectories", color=color,
                     color_continuous_scale=["red", "yellow", "green"],
                     height=600,
                     range_color=(0,0.1), # we can cap the max ratio colour
                     hover_data=["ratio", "functions"]
                    )
    return fig

def show_treemap(df, version):
    treemap(df, version).show()
    
def get_parent(p):
    if p == "kernel root":
        return ""
    if "/" not in p:
        return "kernel root"
    return "/".join(p.split("/")[:-1])


def treemap_with_selector(df):
    selector = widgets.Dropdown(options=[x for x,_,_ in tags()], description="Kernel Version")
    plot = widgets.interactive_output(show_treemap, {'df': fixed(df), 'version':selector})
    return (selector, plot)

In [8]:
display(*treemap_with_selector(non_static_df))

Dropdown(description='Kernel Version', options=('v3.0', 'v3.1', 'v3.2', 'v3.3', 'v3.4', 'v3.5', 'v3.6', 'v3.7'…

Output()

In [9]:
display(*treemap_with_selector(non_static_df))

Dropdown(description='Kernel Version', options=('v3.0', 'v3.1', 'v3.2', 'v3.3', 'v3.4', 'v3.5', 'v3.6', 'v3.7'…

Output()

# https://github.com/Wbec/kernel-test-coverage